In [1]:
%load_ext autoreload

%autoreload 2

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


### 1 - create a SBM graph and extract the edges list


In [2]:
size = 75
s = size
sizes = [s, s, s]
probs = [[0.45, 0.05, 0.02], [0.05, 0.45, 0.07], [0.02, 0.07, 0.40]]
g = nx.stochastic_block_model(sizes, probs, seed=0)
num_nodes = len(g)
edges_list = []
for source, target in g.edges():
    edges_list.append([source, target])

### 2 - Creates a network superactor and adds to a FURY Scene

In [3]:
from helios.fury.actors import FurySuperActorNetwork
from fury import window

colors_by_block = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
colors = np.array([colors_by_block[i//s] for i in range(len(g))]).astype('float64')
markers = [['o', 's', 'd'][i//s]  for i in range(len(g))]

centers = np.random.normal(size=(len(g), 3))


network_super_actor = FurySuperActorNetwork(
        positions=centers,
        colors=colors,
        scales=1,
        edge_width=0,
        marker=markers
)

nodes = network_super_actor.nodes.vtk_actor

scene = window.Scene()
scene.add(nodes)

### 3 - Creates a ShowManager FURY Obj.

In [4]:
from fury import actor, window

window_size = [400, 400]
showm = window.ShowManager(scene, reset_camera=False, size=(
    window_size[0], window_size[1]), order_transparent=False,
)

### 4- Starts the streaming

In [5]:
from fury.stream.widget import Widget

#widget = Widget(showm, encoding='webrtc')
widget = Widget(showm)
widget.start()

created 20 8 1344
url: http://localhost:8315?iframe=1&encoding=mjpeg


In [7]:
widget.return_iframe(300)

### 5 - Start Force-Directed async layout

- ms equals to 100 means that the vtk_positions will be updated at each 100 ms

In [8]:
from helios.core.layout import HeliosFr
layout = HeliosFr(centers,edges_list,showm, network_super_actor)
print('start async helios')
layout.start(100)

start async helios


### 6 - Change some network properties

In [9]:
network_super_actor.nodes.marker_opacity = .5
showm.window.Render()
network_super_actor.nodes.edge_width = .1
network_super_actor.nodes.Uniforms.edgeColor.value = (255, 255, 255)
showm.window.Render()

In [ ]:
layout.stop()

In [ ]:
widget.stop()

In [ ]:
network_super_actor.positions = np.random.normal(size=(num_nodes, 3))
showm.window.Render()